## 一、对数据进行转置df1.T

In [5]:
import pandas as pd

/var/folders/sb/nwf8c4q943g_qwg33z54xpnr0000gn/T/ipykernel_7387/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df1 = pd.read_csv('example4.csv',index_col=0)
df1

,0,1,2
姓名,张三,李四,王五
年龄,30,25,28
地址,北京,上海,广州


In [3]:
df1 = df1.T
df1

,姓名,年龄,地址
0,张三,30,北京
1,李四,25,上海
2,王五,28,广州


## 二、对列进行拆分

In [5]:
df2 = pd.read_csv('example5.csv',index_col=0)
df2

,城市,人口密度
0,城市A,28.53万人/0.87平方公里
1,城市B,37.83万人/2.19平方公里
2,城市C,15.3万人/1.57平方公里


### 1.针对单独的python字符串

split方法，参数可以指定分隔符，得到拆分后的字符串立标

In [6]:
"28.53万人/0.87平方公里".split('/')

['28.53万人', '0.87平方公里']

2.针对series

str.split方法，参数可以指定分隔符，拆分字符串，调用后返回一个新的Series，每一个元素是一个列表。

但这没有达到拆分列的效果，因为Series只能表示DataFrame的一列，而我们希望拆分成多列。

In [7]:
df2['人口密度'].str.split('/')

0    [28.53万人, 0.87平方公里]
1    [37.83万人, 2.19平方公里]
2     [15.3万人, 1.57平方公里]
Name: 人口密度, dtype: object

3.拆分列

仍然使用str.split方法，额外指定一个可选参数expand=True，表示把分隔后的结果分别用单独的Series表示，这样输出结果就会变成一个包含多列的DataFrame

In [9]:
df2['人口密度'].str.split('/',expand=True)

,0,1
0,28.53万人,0.87平方公里
1,37.83万人,2.19平方公里
2,15.3万人,1.57平方公里


### 2.把新生成的dataframe添加进原来的dataframe

In [11]:
df2[['人口','面积']] = df2['人口密度'].str.split('/',expand=True)
df2

,城市,人口密度,人口,面积
0,城市A,28.53万人/0.87平方公里,28.53万人,0.87平方公里
1,城市B,37.83万人/2.19平方公里,37.83万人,2.19平方公里
2,城市C,15.3万人/1.57平方公里,15.3万人,1.57平方公里


### 3.删除拆分前的列

In [12]:
df2 = df2.drop('人口密度',axis=1)
df2

,城市,人口,面积
0,城市A,28.53万人,0.87平方公里
1,城市B,37.83万人,2.19平方公里
2,城市C,15.3万人,1.57平方公里


## 三、把不同列合并成一列

In [16]:
df3 = pd.read_csv('example6.csv',index_col=0)
df3

,姓,名,年龄
0,张,三,30
1,李,四,25
2,王,五,35


In [19]:
df3['姓名'] = df3['姓'].str.cat(df3['名'])
df3

,姓,名,年龄,姓名
0,张,三,30,张三
1,李,四,25,李四
2,王,五,35,王五


In [20]:
df3 = df3.drop(['姓','名'], axis=1)
df3

,年龄,姓名
0,30,张三
1,25,李四
2,35,王五


### 四、把宽数据转换成长数据
即重塑列，具体来说是在把一部分列名，转换为变量，pd.melt方法，可以帮我们进行这方面的转换。

In [7]:
df4 = pd.read_csv('example7.csv')
df4

,国家代码,年份,男性年龄组（0-4岁）,男性年龄组（5-14岁）,男性年龄组（15-24岁）,女性年龄组（0-4岁）,女性年龄组（5-14岁）,女性年龄组（15-24岁）
0,CN,2019,100,200,500,80,150,400
1,US,2019,50,150,300,40,100,200
2,JP,2019,30,120,250,20,80,150
3,IN,2019,80,180,400,60,120,300



第一个参数传入要转换的DataFrame

可选参数id_vars传入一个列表，里面放入想保持原样的列，那么除了id_vars列表里面之外的列，都会被视为要进行转换的列

可选参数var_name，要被赋值为在转换后的DataFrame中，包含原本列名值的新列列名

可选参数value_name，要被赋值为在转换后的DataFrame中，包含原本变量值的新列列名

运行后，就会返回一个从原本宽数据转换成长数据的DataFrame

In [8]:
pd.melt(df4,id_vars=['国家代码','年份'],var_name='年龄组',value_name='肺结核病例数')

,国家代码,年份,年龄组,肺结核病例数
0,CN,2019,男性年龄组（0-4岁）,100
1,US,2019,男性年龄组（0-4岁）,50
2,JP,2019,男性年龄组（0-4岁）,30
3,IN,2019,男性年龄组（0-4岁）,80
4,CN,2019,男性年龄组（5-14岁）,200
5,US,2019,男性年龄组（5-14岁）,150
6,JP,2019,男性年龄组（5-14岁）,120
7,IN,2019,男性年龄组（5-14岁）,180
8,CN,2019,男性年龄组（15-24岁）,500
9,US,2019,男性年龄组（15-24岁）,300


注意：这里并未改变原始dataframe，如需改变生效的话，可以重新复制。

### 五、对行进行拆分

df.explode方法，传入需要拆分的变量名，就会返回一个把该列列表中每一个元素，转换为单独一行的新DataFrame

这个方法很实用，当每个单元格是一个值而不是列表时，更好进行统计

In [55]:
df5 = pd.read_csv('example8.csv')
df5

,学生姓名,学号,课程列表
0,张三,1,"['数学','物理']"
1,李四,2,"['英语','化学']"
2,王五,3,"['语文','数学','英语','政治']"
3,赵六,4,"['物理','生物']"


这里的课程列表的各个值是字符串，不是列表形式，需要先将字符串转换为列表，才能使用explode进行拆分。

In [56]:
print(df5['课程列表'].apply(type))

0    <class 'str'>
1    <class 'str'>
2    <class 'str'>
3    <class 'str'>
Name: 课程列表, dtype: object


In [57]:
import ast
df5['课程列表'] = df5['课程列表'].apply(ast.literal_eval) 

In [58]:
df5 = df5.explode('课程列表')
df5

,学生姓名,学号,课程列表
0,张三,1,数学
0,张三,1,物理
1,李四,2,英语
1,李四,2,化学
2,王五,3,语文
2,王五,3,数学
2,王五,3,英语
2,王五,3,政治
3,赵六,4,物理
3,赵六,4,生物


### 六、对行或者列进行删除

使用drop方法


基本上，对DataFrame的操作方法，都是默认不改变原始DataFrame的，而是返回一个新的DataFrame

要么就得进行重新赋值，要么指定可选参数inplace=True